In [1]:
import os
import pandas as pd


In [2]:
filepath = "Resources/MUP_INP_RY24_P03_V10_DY22_PrvSvc.csv"
codefilepath = "Resources/DRG_to_MDC_Crosswalk.xlsx"
rucafilepath = "Resources/ruca_codes.csv"
countyfilepath = "Resources/county_zip_crosswalk.csv"

fulldata = pd.read_csv(filepath, encoding='ISO-8859-1')
codedata = pd.read_excel(codefilepath)
rucadata = pd.read_csv(rucafilepath)

fulldata.head()

,Rndrng_Prvdr_CCN,Rndrng_Prvdr_Org_Name,Rndrng_Prvdr_City,Rndrng_Prvdr_St,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,DRG_Cd,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,1.0,Metropolitan area core: primary flow within an...,23,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,25,158541.640000,37331.000000,35332.960000
1,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,1.0,Metropolitan area core: primary flow within an...,24,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,18,107085.333330,25842.666667,23857.944444
2,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,1.0,Metropolitan area core: primary flow within an...,25,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCE...,18,156326.777780,32167.888889,27662.944444
3,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,1.0,Metropolitan area core: primary flow within an...,38,EXTRACRANIAL PROCEDURES WITH CC,19,112085.263160,11568.473684,9993.473684
4,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,1.0,Metropolitan area core: primary flow within an...,39,EXTRACRANIAL PROCEDURES WITHOUT CC/MCC,33,89068.212121,8199.818182,6086.393939


In [3]:
validdata = fulldata.drop(fulldata.columns[[0,1,2,3,4,8,10]], axis=1)


In [4]:
validdata = validdata.rename(columns={
    'Rndrng_Prvdr_Zip5': 'zip',
    'Rndrng_Prvdr_State_Abrvtn': 'state',
    'Rndrng_Prvdr_RUCA': 'RUCA',
    'Tot_Dschrgs': 'discharges',
    'Avg_Submtd_Cvrd_Chrg': 'avg_covered_charge',
    'Avg_Tot_Pymt_Amt': 'avg_total_payment',
    'Avg_Mdcr_Pymt_Amt': 'avg_medicare_payment',
})
validdata.head()

,zip,state,RUCA,DRG_Cd,discharges,avg_covered_charge,avg_total_payment,avg_medicare_payment
0,36301,AL,1.0,23,25,158541.640000,37331.000000,35332.960000
1,36301,AL,1.0,24,18,107085.333330,25842.666667,23857.944444
2,36301,AL,1.0,25,18,156326.777780,32167.888889,27662.944444
3,36301,AL,1.0,38,19,112085.263160,11568.473684,9993.473684
4,36301,AL,1.0,39,33,89068.212121,8199.818182,6086.393939


In [5]:

validdata['avg_oop'] = validdata['avg_total_payment']-validdata['avg_medicare_payment']

validdata.head()

,zip,state,RUCA,DRG_Cd,discharges,avg_covered_charge,avg_total_payment,avg_medicare_payment,avg_oop
0,36301,AL,1.0,23,25,158541.640000,37331.000000,35332.960000,1998.040000
1,36301,AL,1.0,24,18,107085.333330,25842.666667,23857.944444,1984.722223
2,36301,AL,1.0,25,18,156326.777780,32167.888889,27662.944444,4504.944445
3,36301,AL,1.0,38,19,112085.263160,11568.473684,9993.473684,1575.000000
4,36301,AL,1.0,39,33,89068.212121,8199.818182,6086.393939,2113.424242


In [6]:
validdata = validdata.dropna(subset=['RUCA'])
validdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145065 entries, 0 to 145741
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   zip                   145065 non-null  int64  
 1   state                 145065 non-null  object 
 2   RUCA                  145065 non-null  float64
 3   DRG_Cd                145065 non-null  int64  
 4   discharges            145065 non-null  int64  
 5   avg_covered_charge    145065 non-null  float64
 6   avg_total_payment     145065 non-null  float64
 7   avg_medicare_payment  145065 non-null  float64
 8   avg_oop               145065 non-null  float64
dtypes: float64(5), int64(3), object(1)
memory usage: 11.1+ MB


In [7]:
#Prepare DRG to MDC code conversion file for merging by renaming DRG Code column 
codedata = codedata.rename(columns={"DRG_CODE":"DRG_Cd"})

#Merge validdata set with coding dataset to assign MDC codes to each column
alldata= pd.merge(codedata,validdata, how="left", on='DRG_Cd')
alldata.head()

,DRG_Cd,MDC_CODE,Description,zip,state,RUCA,discharges,avg_covered_charge,avg_total_payment,avg_medicare_payment,avg_oop
0,1,DRG 001,Heart transplant or implant of heart assist sy...,85054.0,AZ,1.0,14.0,9.540479e+05,452565.57143,191405.21429,261160.35714
1,1,DRG 001,Heart transplant or implant of heart assist sy...,72205.0,AR,1.0,20.0,1.242436e+06,268918.15000,236840.90000,32077.25000
2,1,DRG 001,Heart transplant or implant of heart assist sy...,92103.0,CA,1.0,32.0,9.801762e+05,350290.68750,322569.34375,27721.34375
3,1,DRG 001,Heart transplant or implant of heart assist sy...,95816.0,CA,1.0,11.0,1.319050e+06,375472.45455,309435.54545,66036.90910
4,1,DRG 001,Heart transplant or implant of heart assist sy...,94305.0,CA,1.0,22.0,3.767105e+06,501627.09091,478453.95455,23173.13636


In [8]:
alldata.dtypes


DRG_Cd                    int64
MDC_CODE                 object
Description              object
zip                     float64
state                    object
RUCA                    float64
discharges              float64
avg_covered_charge      float64
avg_total_payment       float64
avg_medicare_payment    float64
avg_oop                 float64
dtype: object

In [9]:
#Merge rucadata set with alldata dataset to assign MDC codes to each column
alldata= pd.merge(rucadata,alldata, how="left", on='RUCA')

alldata.head()

,RUCA,RUCA_Des,c_type,DRG_Cd,MDC_CODE,Description,zip,state,discharges,avg_covered_charge,avg_total_payment,avg_medicare_payment,avg_oop
0,1.0,Metropolitan area core: primary flow within an...,Urban,1,DRG 001,Heart transplant or implant of heart assist sy...,85054.0,AZ,14.0,9.540479e+05,452565.57143,191405.21429,261160.35714
1,1.0,Metropolitan area core: primary flow within an...,Urban,1,DRG 001,Heart transplant or implant of heart assist sy...,72205.0,AR,20.0,1.242436e+06,268918.15000,236840.90000,32077.25000
2,1.0,Metropolitan area core: primary flow within an...,Urban,1,DRG 001,Heart transplant or implant of heart assist sy...,92103.0,CA,32.0,9.801762e+05,350290.68750,322569.34375,27721.34375
3,1.0,Metropolitan area core: primary flow within an...,Urban,1,DRG 001,Heart transplant or implant of heart assist sy...,95816.0,CA,11.0,1.319050e+06,375472.45455,309435.54545,66036.90910
4,1.0,Metropolitan area core: primary flow within an...,Urban,1,DRG 001,Heart transplant or implant of heart assist sy...,94305.0,CA,22.0,3.767105e+06,501627.09091,478453.95455,23173.13636


In [10]:
alldata= alldata.drop(alldata.columns[[0,1,3]], axis=1)
alldata = alldata.rename(columns={"Description":"mdc_desc"})
alldata.head()

,c_type,MDC_CODE,mdc_desc,zip,state,discharges,avg_covered_charge,avg_total_payment,avg_medicare_payment,avg_oop
0,Urban,DRG 001,Heart transplant or implant of heart assist sy...,85054.0,AZ,14.0,9.540479e+05,452565.57143,191405.21429,261160.35714
1,Urban,DRG 001,Heart transplant or implant of heart assist sy...,72205.0,AR,20.0,1.242436e+06,268918.15000,236840.90000,32077.25000
2,Urban,DRG 001,Heart transplant or implant of heart assist sy...,92103.0,CA,32.0,9.801762e+05,350290.68750,322569.34375,27721.34375
3,Urban,DRG 001,Heart transplant or implant of heart assist sy...,95816.0,CA,11.0,1.319050e+06,375472.45455,309435.54545,66036.90910
4,Urban,DRG 001,Heart transplant or implant of heart assist sy...,94305.0,CA,22.0,3.767105e+06,501627.09091,478453.95455,23173.13636


In [11]:
alldata = alldata.rename(columns={"MDC_CODE":"mdc_code"})
newcolumnorder= ['state','zip','c_type','mdc_code','mdc_desc','discharges','avg_covered_charge','avg_total_payment','avg_medicare_payment','avg_oop']
alldata = alldata[newcolumnorder]
alldata['zip'] = alldata['zip'].astype(int)
alldata['discharges'] = alldata['discharges'].astype(int)
alldata.head()

,state,zip,c_type,mdc_code,mdc_desc,discharges,avg_covered_charge,avg_total_payment,avg_medicare_payment,avg_oop
0,AZ,85054,Urban,DRG 001,Heart transplant or implant of heart assist sy...,14,9.540479e+05,452565.57143,191405.21429,261160.35714
1,AR,72205,Urban,DRG 001,Heart transplant or implant of heart assist sy...,20,1.242436e+06,268918.15000,236840.90000,32077.25000
2,CA,92103,Urban,DRG 001,Heart transplant or implant of heart assist sy...,32,9.801762e+05,350290.68750,322569.34375,27721.34375
3,CA,95816,Urban,DRG 001,Heart transplant or implant of heart assist sy...,11,1.319050e+06,375472.45455,309435.54545,66036.90910
4,CA,94305,Urban,DRG 001,Heart transplant or implant of heart assist sy...,22,3.767105e+06,501627.09091,478453.95455,23173.13636


In [12]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142893 entries, 0 to 142892
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   state                 142893 non-null  object 
 1   zip                   142893 non-null  int32  
 2   c_type                142893 non-null  object 
 3   mdc_code              142893 non-null  object 
 4   mdc_desc              142893 non-null  object 
 5   discharges            142893 non-null  int32  
 6   avg_covered_charge    142893 non-null  float64
 7   avg_total_payment     142893 non-null  float64
 8   avg_medicare_payment  142893 non-null  float64
 9   avg_oop               142893 non-null  float64
dtypes: float64(4), int32(2), object(4)
memory usage: 9.8+ MB


In [13]:
alldata.to_csv('cleanedmedicaredata.csv', index=False)